Sveučilište u Zagrebu<br>
Fakultet elektrotehnike i računarstva

# Strojno učenje

<a href="http://www.fer.unizg.hr/predmet/su">http://www.fer.unizg.hr/predmet/su</a>

Ak. god. 2015./2016.

# Bilježnica 8: Stroj potpornih vektora (SVM)

(c) 2015 Jan Šnajder

<i>Verzija: 0.1 (2015-12-08)</i>

In [2]:
import scipy as sp
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Sadržaj:

* Uvod

* Problem maksimalne margine

* Metoda Lagrangeovih multiplikatora

* Dualna formulacija problema maksimalne margine

* Meka margina

* Gubitak zglobnice

* Jezgreni trik

* Mercerove jezgre

* Optimizacija hiperparametara


# Uvod


* Vrlo učinkovit **diskriminativan model**


* Podsjetnik: Algoritam strojnog učenja definiran je 
  1. modelom
  * pogreškom
  * optimizacijskim postupkom
    
    
* Model:
    
$$
    h(\mathbf{x}) = \mathbf{w}^\intercal\boldsymbol{\phi}(\mathbf{x})
$$


* Dakle, to je poopćeni linearan model bez funkcije preslikavanja


* Gornja definicija modela je tzv. **primarna formulacija**


* Postoji i **dualna formulacija**:
  * Umjesto da značajke novog primjera množimo težinama $\mathbf{w}$, možemo za novi primjer izračunati koliko je sličan primjerima iz skupa za učenje i na temelju toga odrediti klasifikaciju
  * Model efektivno postaje **neparametarski**!
  
  
* U dualnoj formulaciji možemo iskoristiti tzv. **jezgreni trik**, koji nam omogućava jeftino preslikavanje primjera u prostor više dimenzije (a time i nelinearnost modela)
  * SVM je jedna vrsta tzv. **jezgrenog stroja** (engl. *kernel machine*)


* Model je jednostavan, no definicija pogreške i optimizacijskog postupka su nešto složeniji


* Osnovna ideja: primjere dviju klasa razdvojiti tako da je prostor između njih što veći $\Rightarrow$ tzv. **maksimalna margina**


* Opravdanje: generalizacija će biti najbolja onda kada granicu između klasa povućemo točno po sredini margine


* Do sada smo optimizacijski problem definirali tako da smo
  * krenuli od log-izglednosti pa izveli MLE (Naivan bayes)
  * krenuli od funkcije gubitka pa izveli funkciju pogreške (linearna regresija) i napravili analitičku minimizaciju
  * krenuli od funkcije pogreške pa izveli funkciju gubitka (logistička regresija, perceptron) i iskoristili je za gradijentni spust
  
  
* Kod SVM-a, krenut ćemo odmah od onoga što u konačnici želimo dobiti: maksimalnu marginu $\Rightarrow$ to je ono što SVM optimizira


* Naknadno ćemo iz toga izvesti funkciju gubitka (i funkciju pogreške), ali samo radi usporedbe s drugim algoritmima


> **Sinopsis:**
> 
> * Prvo ćemo se fokusirati na **linearan model** i **linearno odvojive probleme**
  * Matematika: Lagrangeovi multiplikatori
> 
> * Zatim ćemo proširiti **linearan model** tako da može raditi s **linearno neodvojivim problemima** (meka granica)
> 
> * Na kraju ćemo proširiti na **nelinearan model** (jezgreni trik)
>   * Matematika: Mercerove jezgre

# Problem maksimalne margine

* Model:
$$
h(\mathbf{x}) = \mathbf{w}^\intercal x + w_0
$$


* Oznake primjera za učenje: $y\in\{-1,+1\}$


* Granica između klasa: hiperravnina $h(\mathbf{x})=0$


* Predikcija klase: $y=\mathrm{sgn}(h(\mathbf{x}))$


* Pretpostavimo da su primjeri iz $\mathcal{D}$ **linearno odvojivi**


* Onda postoji $\mathbf{w}$ i $w_0$ takvi da
$$
\begin{align*}
h(\mathbf{x}^{(i)}) \geq 0 & \quad\text{za $y^{(i)}=+1$}\\
h(\mathbf{x}^{(i)}) < 0 & \quad\text{za $y^{(i)}=-1$}\\
\end{align*}
$$


* Kraće, postoje $\mathbf{w}$ i $w_0$ takvi da
$$
\forall(\mathbf{x}^{(i)},y^{(i)}) \in D.\ y^{(i)}h(\mathbf{x}^{(i)})\geq 0
$$


* Postoji beskonačno mnogo rješenja za $\mathbf{w}$ i $w_0$ (prostor inačica je beskonačan)


* No nas zanima rješenje **maksimalne margine** $\Rightarrow$ induktivna pristranost preferencijom


* **Margina = udaljenost hiperravnine do najbližeg primjera**


* Ako maksimiziramo marginu, onda će hiperravnina prolaziti točno kroz na pola puta između dva primjera


#### Formulacija optimizacijskog problema


* Predznačena udaljenost primjera od hiperravnine je

$$
d = \frac{h(\mathbf{x})}{\|\mathbf{w}\|}
$$


* Nas zanimaju samo hiperravnine koje ispravno klasificiraju primjere. U tom slučaju **apsolutna** udaljenost primjera do hiperravnine je:

$$
\frac{y^{(i)}h(\mathbf{x})}{\|\mathbf{w}\|} = 
\frac{y^{(i)}(\mathbf{w}^\intercal\mathbf{x}^{(i)}+w_0)}{\|\mathbf{w}\|}
$$


* Po definiciji, margina je udaljenost hiperravnine do najbližeg primjera:

$$
\frac{1}{\|\mathbf{w}\|}\mathrm{min}_i\big\{y^{(i)}(\mathbf{w}^\intercal\mathbf{x} + w_0)\big\}
$$


* Tu udaljenost želimo maksimizirati:

$$
\mathrm{argmax}_{\mathbf{w},w_0}\Big\{\frac{1}{\|\mathbf{w}\|}\mathrm{min}_i\big\{y^{(i)}(\mathbf{w}^\intercal\mathbf{x} + w_0)\big\}\Big\}
$$


* Ako je $\mathcal{D}$ linearno odvojiv, onda postoji samo jedna takva margina

#### Pojednostavljenje optimizacijskog problema


* Gornji problem teško je riješiti izravno (min unutar max)


* Vektor $(\mathbf{w},w_0)$ možemo pomnožiti s proizvoljnom konstantom, a da to ne utječe na udaljenosti između primjera i hiperravnine:

$$
d=\frac{h(\mathbf{x})}{\|\mathbf{w}\|}=
\frac{\color{red}{\alpha}\mathbf{w}^\intercal\mathbf{x}+\color{red}{\alpha}w_0}{\|\color{red}{\alpha}\mathbf{w}\|}=
\frac{\color{red}{\alpha}(\mathbf{w}^\intercal\mathbf{x}+w_0)}{\color{red}{\alpha}\|\mathbf{w}\|}=
\frac{\mathbf{w}^\intercal\mathbf{x}+w_0}{\|\mathbf{w}\|}
$$


* Kako bismo pojednostavili problem, možemo definirati da za primjer $\mathbf{x}^{(i)}$ koji je najbliži margini vrijedi

$$
y^{(i)}(\mathbf{w}^\intercal\mathbf{x}+w_0)=1
$$


* Svi ostali primjeri jednako su blizu margine ili su od nje još udaljeniji:

$$
y^{(i)}(\mathbf{w}^\intercal\mathbf{x}+w_0) \geq 1, \qquad n=1,\dots,N
$$


* Za primjere za koje $y^{(i)}h(\mathbf{x})=1$ kažemo da su ograničenja **aktivna**, dok su za ostale primjere ograničenja neaktivna 
  * (Primjeri za koje su ograničenja aktivna zovemo potpornim vektorima, ali o tome više kasnije)


* Uvijek će postojati barem **dva** aktivna ograničenja


* [Skica: maksimalna margina]


* Dakle, umjesto
$$
\mathrm{argmax}_{\mathbf{w},w_0}\Big\{\frac{1}{\|\mathbf{w}\|}\underbrace{\mathrm{min}_i\big\{y^{(i)}(\mathbf{w}^\intercal\mathbf{x} + w_0)\big\}}_{=1}\Big\}
$$
mi sada maksimiziramo
$$
\mathrm{argmax}_{\mathbf{w},w_0}\frac{1}{\|\mathbf{w}\|}
$$
uz ograničenja
$$
y^{(i)}(\mathbf{w}^\intercal\mathbf{x}+w_0) \geq 1, \qquad n=1,\dots,N
$$


* Maksimizator od $\frac{1}{\|\mathbf{w}\|}$ ekvivelentan je minimizatoru od $\|\mathbf{w}\|=\sqrt{\mathbf{w}^\intercal\mathbf{w}}$, a taj je ekvivalentan minimizatoru od $\|\mathbf{w}\|^2$. Još ćemo pomnožiti s $\frac{1}{2}$ radi kasnije matematičke jednostavnosti


* Konačna formulacija optimizacijskog problema

> $\mathrm{argmin}_{\mathbf{w},w_0}\frac{1}{2}\|\mathbf{w}\|^2$

> tako da $\quad y^{(i)}(\mathbf{w}^\intercal\mathbf{x}+w_0) \geq 1, \quad n=1,\dots,N$


* Naš optimizacijski problem sveo se na ciljnu funkciju koju želimo optimirati i ograničenja koja pritom moramo poštovati: tipičan problem **konveksne optimizacije uz ograničenja**



# Metoda Lagrangeovih multiplikatora

* Problem **kvadratnog programiranja** (kvadratno ograničenog kvadratnog programiranja, QCQP)


* Može se riješiti metodom **Langrangeovih multiplikatora**

### Metoda Lagrangeovih multiplikatora


### Lagrangeova dualnost



# Dualna formulacija problema maksimalne margine



* Lagrangeova funkcija za naš problem:
$$
L(\mathbf{w},w_0,\color{red}{\boldsymbol\alpha})=\frac{1}{2}\|\mathbf{w}\|^2 -
\sum_{i=1}^N\color{red}{\alpha_i}\Big\{y^{(i)}\big(\mathbf{w}^\intercal\mathbf{x}^{(i)}+w_0\big)-1\Big\}
$$


*  $\color{red}{\boldsymbol\alpha=(\alpha_1,\dots,\alpha_N)}$ je vektor Lagrangeovih multiplikatora, po jedan za svako ograničenje


* Prelazimo na **dualnu formulaciju** problema jer je ta formulacija jednostavnija


* Minimizator $(\mathbf{w}^*, w_0^*)$: deriviranje po $\mathbf{w}$ odnosno $w_0$ i izjednačavanje s nulom:
$$
\begin{align}
\mathbf{w} &= \sum_{i=1}^N \alpha_i y^{(i)}\mathbf{x}^{(i)}\\
0 &= \sum_{i=1}^N\alpha_i y^{(i)}
\end{align}
$$


* Dualna Lagrangeova funkcija:

$$
\begin{align*}
\tilde{L}(\boldsymbol\alpha) &=
\frac{1}{2}\|\mathbf{w}\|^2 -\sum_{i=1}^N\alpha_i\Big\{y^{(i)}\big(\mathbf{w}^\intercal\mathbf{x}^{(i)}+w_0\big)-1\Big\}\\
&= 
\frac{1}{2}\|\mathbf{w}\|^2
-\sum_{i=1}^N\alpha_i y^{(i)}\mathbf{w}^\intercal\mathbf{x}^{(i)}
\color{gray}{\underbrace{-w_0 \sum_{i=1}^N\alpha_iy^{(i)}}_{=0}} + 
\sum_{i=1}^N\alpha_i\\
&= \nonumber
\frac{1}{2}\sum_{i=1}^N\alpha_i y^{(i)}(\mathbf{x}^{(i)})^\intercal\sum_{j=1}^N\alpha_j y^{(j)}\mathbf{x}^{(j)}
-
\sum_{i=1}^N\alpha_i y^{(i)}(\mathbf{x}^{(i)})^\intercal\sum_{j=1}^N\alpha_j y^{(j)}\mathbf{x}^{(j)}
+ \sum_{i=1}^N\alpha_i\\
&= 
\sum_{i=1}^N\alpha_i - 
\frac{1}{2}\sum_{i=1}^N
\sum_{j=1}^N
\alpha_i
\alpha_j
y^{(i)}
y^{(j)}
(\mathbf{x}^{(i)})^\intercal
\mathbf{x}^{(j)}
\end{align*}
$$

* Dobili smo **dualni optimizacijski problem**:

> **Maksimizirati** izraz
> \begin{align}
 \sum_{i=1}^N\alpha_i - 
 \frac{1}{2}\sum_{i=1}^N
 \sum_{j=1}^N
 \alpha_i
 \alpha_j
 y^{(i)}
 y^{(j)}
 (\mathbf{x}^(i))^\intercal
 (\mathbf{x}^(j))
 \end{align}
> tako da:
> \begin{align*}
 \alpha_i &\geq 0,\quad i=1,\dots,N \\ 
 \sum_{i=1}^N\alpha_i y^{(i)} &= 0
 \end{align*}



# Sažetak

* Logistička regresija je **diskriminativan klasifikacijski model** s probabilističkim izlazom
  
  
* Koristi se **logistička funkcija gubitka** odnosno **pogreška unakrsne entropije**


* Optimizacija se provodi **gradijentnim spustom**, a prenaučenost se može spriječiti **regularizacijom**


* Model **odgovara generativnom modelu** s normalno distribuiranim izglednostima i dijeljenom kovarijacijskom matricom, ali je broj parametara logističke regresije manji 


* Logistička regresija vrlo je dobar algoritam koji **nema        nedostatke** koje imaju klasifikacija regresijom i perceptron
